In [ ]:
!pip install selenium
!pip install beautifulsoup4
!pip install chromedriver-autoinstaller

  Using cached urllib3-1.26.14-py2.py3-none-any.whl (140 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.22
    Uninstalling urllib3-1.22:
      Successfully uninstalled urllib3-1.22


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.18.4 requires urllib3<1.23,>=1.21.1, but you have urllib3 1.26.14 which is incompatible.
pandas-profiling 3.6.1 requires requests<2.29,>=2.24.0, but you have requests 2.18.4 which is incompatible.
cookiecutter 1.7.3 requires requests>=2.23.0, but you have requests 2.18.4 which is incompatible.
conda 22.11.1 requires requests>=2.20.1, but you have requests 2.18.4 which is incompatible.
conda-repo-cli 1.0.24 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.24 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.24 requires requests==2.28.1, but you have requests 2.18.4 which is incompatible.
anaconda-client 1.11.0 requires requests>=2.20.0, but you have requests 2.18.4 which is incompatible.


# 1. 수집에 필요한 라이브러리 불러오기

In [11]:
import os, time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib.request
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
from tqdm import tqdm
import chromedriver_autoinstaller
from selenium import webdriver
from tqdm import tqdm
import numpy as np
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup

In [12]:
import os, time
from selenium import webdriver
import urllib.request
from selenium.webdriver.common.by import By

## list -> 문자열로 변경함수

In [177]:
# list -> 문자열로 변경함수
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

# 2.webdriver 이용하여 melon 사이트와 연결

In [ ]:
driver = webdriver.Chrome()

In [14]:
driver.get(f'https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate=1990')


In [20]:
music_list = []
for i in tqdm(range(1960,2024)):
    driver.get(f'https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate={i}')
    for j in driver.find_elements(By.CSS_SELECTOR , '#lst50 > td:nth-child(4) > div > button.btn_icon.play'):
        music_list.append(j.get_attribute('onclick').split(',')[1].split(')')[0])

100%|██████████████████████████████████████████████████████████████████████████████████| 64/64 [00:46<00:00,  1.38it/s]


In [21]:
music_list2 = []
for i in tqdm(range(1960,2024)):
    driver.get(f'https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate={i}#params%5Bidx%5D=51')
    for j in driver.find_elements(By.CSS_SELECTOR , '#lst100 > td:nth-child(4) > div > button.btn_icon.add'):
        music_list2.append(j.get_attribute('onclick').split('(')[1].split(')')[0])

100%|██████████████████████████████████████████████████████████████████████████████████| 64/64 [00:37<00:00,  1.72it/s]


In [22]:
Top100_list = music_list + music_list2

In [23]:
Top100_list = list(set(Top100_list))

In [296]:
driver.get("https://www.melon.com/song/detail.htm?songId=1415517")

for i in driver.find_elements(By.CSS_SELECTOR , '#downloadfrm > div > div > div.entry > div.info > div.song_name'):
    print(i.text)
    print(type(i.text))

The Day
<class 'str'>


# 2. 이미지, 제목, 가수, 발매일, 장르, 가사 크롤링해오기

In [26]:
img = []
title = []
singer = []
release_date = []
genre = []
lyrics = []
for i in tqdm(Top100_list):
    time.sleep(2)
    driver.get(f"https://www.melon.com/song/detail.htm?songId={i}")
    # 이미지 
    
    for i in driver.find_elements(By.CSS_SELECTOR , '#downloadfrm > div > div > div.thumb > a > img'):
        if i.text == None:
            img.append('null')
        else:
            img.append(i.get_attribute('src'))
        
    # 제목
    
    for i in driver.find_elements(By.CSS_SELECTOR , '#downloadfrm > div > div > div.entry > div.info > div.song_name'):
        if i.text == None:
            title.append('null')
        else:
            title.append(i.text)
    #가수
    
    for i in driver.find_elements(By.CSS_SELECTOR , '#downloadfrm > div > div > div.entry > div.info > div.artist'):
        if i.text == None:
            singer.append('null')
        else:
            singer.append(i.text)
            
    #발매일
    for i in driver.find_elements(By.CSS_SELECTOR , '#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)'):
        if i.text == None:
            release_date.append('null')
        else:
            release_date.append(i.text)
            
    #장르
    for i in driver.find_elements(By.CSS_SELECTOR , '#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)'):
        if i.text == None:
            genre.append('null')
        else:
            genre.append(i.text)
    #가사
    
    for i in driver.find_elements(By.CSS_SELECTOR , '#lyricArea > div'):
        if i.text == None:
            lyrics.append('null')
        else:
            lyrics.append(i.text)
    
music = pd.DataFrame(zip(title, singer, release_date, genre,lyrics, img))

100%|████████████████████████████████████████████████████████████████████████████| 4412/4412 [3:25:09<00:00,  2.79s/it]


In [27]:
music.columns = ['제목', '가수', '발매일','장르','가사', '이미지']

In [29]:
music

,제목,가수,발매일,장르,가사,이미지
0,날 사랑했다면,박학기,1993.10.01,발라드,떠나버린 널 깨닫기엔\n하늘이 너무 맑아\n우리에게 필요한 건 시간일뿐\n\n영화속...,https://cdnimg.melon.co.kr/cm/album/images/005...
1,그대 떠난 이밤에,옥슨 80,1981.09.25,록/메탈,바람부네 아픈 내맘에\n아름다운 추억 못잊어 부네\n그대 떠난 이밤\n그대 떠난 이...,https://cdnimg.melon.co.kr/cm/album/images/000...
2,나는 홀로 있어도,유심초,1980.11.01,성인가요/트로트,하얀 불빛 아래에\n침묵만이 흐르고\n낯설은 네 눈길에\n눈물만이 흐르네\n멀어져 ...,https://cdnimg.melon.co.kr/cm/album/images/003...
3,Love Is (Dance Mega Mix Ver.),터보,2001.06.20,댄스,우린 6년전에 만났지\n\n널 사랑하게 됐어\n\n내 마음을 숨긴체\n\n널 따라 ...,https://cdnimg.melon.co.kr/cm/album/images/003...
4,사랑밖엔 난 몰라 (Original Ver.),심수봉,2009.08.13,성인가요/트로트,그대 내 곁에선 순간\n\n그 눈빛이 너무 좋아\n\n어제는 울었지만\n\n오늘은 ...,https://cdnimg.melon.co.kr/cm/album/images/006...
...,...,...,...,...,...,...
4406,작은 기다림,쿨 (COOL),1995.10.01,댄스,널 이렇게 보내 줄 수\n밖에 없었어\n나 후회할 지도 모른 채\n끝없이 다가오는 ...,https://cdnimg.melon.co.kr/cm/album/images/000...
4407,WE LIKE 2 PARTY,BIGBANG (빅뱅),2015.06.01,랩/힙합,오늘도 친구들이 왔어\nMAN HOW YOU BEEN WHATS UP\nAYE 여기...,https://cdnimg.melon.co.kr/cm/album/images/023...
4408,내 손을 잡아,아이유,2011.05.25,"발라드, 국내드라마",느낌이 오잖아\n\n떨리고 있잖아\n\n언제까지 눈치만 볼 거니\n\n네 맘을 말해...,https://cdnimg.melon.co.kr/cm/album/images/012...
4409,Lie (Radio Ver.),박화요비,2000.06.14,R&B/Soul,말해줄래 나를 위해라면\n이러진 말아\n아직 그댈 놓아주기에는\n너무 난 두려워\n...,https://cdnimg.melon.co.kr/cm/album/images/000...


# 3.음악정보들을 music csv로 저장

In [30]:
music.to_csv('music.csv',index=False,encoding="utf-8-sig")

# 4. 이미지 html -> jpg로 변환 후 img 폴더에 저장

In [31]:
import urllib.request

In [48]:
music['이미지'][1]

'https://cdnimg.melon.co.kr/cm/album/images/000/50/313/50313_500.jpg/melon/resize/282/quality/80/optimize'

In [1]:
# 이미지 html -> jpg로 변환
import requests

for index, i in enumerate(music['이미지']):
    response = requests.get(i)

    with open(f"img/{index}img.jpg", "wb") as f:
        f.write(response.content)

NameError: name 'music' is not defined